In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("Advertising.csv")
df = df.iloc[:,1:]
df.head()

,TV,Radio,Newspaper,Sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [4]:
features = df[['TV', 'Radio', 'Newspaper']]
response = df['Sales']
total_ads = pd.DataFrame(features.sum(axis=1), columns=['Total Ads'])
total_ads.head()

,Total Ads
0,337.1
1,128.9
2,132.4
3,251.3
4,250.0


In order to determine the relationship between total ads and sales, I calculate the pearson correlation coefficient.

In [5]:
def pearson_correlation(x, y):
    # Calculate the means
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    
    # Calculate the differences from the means
    diff_x = x - mean_x
    diff_y = y - mean_y
    
    # Calculate the covariance and standard deviations
    covariance = np.sum(diff_x * diff_y)
    std_x = np.sqrt(np.sum(diff_x**2))
    std_y = np.sqrt(np.sum(diff_y**2))
    
    # Calculate the correlation coefficient
    correlation = covariance / (std_x * std_y)
    
    return correlation

In [6]:
pearson_correlation(total_ads["Total Ads"], response)

0.867712302701742

Ex 4 b)

The correlation coefficient r = 0.87 is positive and greater than 0.7. Therefore we can assume a strong positive relationship between total ads and sales. So: As the total ad expenses increase, the sales also tend to increase.

Furthermore I will perform a linear regression analysis using MLE:
As shown in the lecture the coefficient can be derived by the following formula:

In [7]:
def fit_model(X, Y):
    XTX = np.dot(X.T, X)
    XTY = np.dot(X.T, Y)
    beta = np.dot(np.linalg.inv(XTX), XTY)

    return beta

In [9]:
X = total_ads[['Total Ads']]
Y = df['Sales']
X.insert(0, 'Intercept', 1)

In [10]:
fit_model(X,Y)

array([4.24302822, 0.04868788])

Also the estimate coefficient for TV ad expense is positive. 

Interpretation: 
If we would not spent any money on ads, we would sell around 4.243028 units

If we increase our ad expenses by 1 unit, our sales would increase by 0.048688.

Or state it like that: An additional $1,000 spent ads is associated with an increase in sales of 48.688 sales.

So we find a postive relationship between ad expenses and sales.

Ex. 4 c)

In order to find the strongest and weakest relationship between the different types of ads and sales. We calculate the correlation between each of them

In [12]:
for ad in features.columns.tolist():
    print(ad + ": r = " + str(pearson_correlation(features[ad], response)))

TV: r = 0.7822244248616063
Radio: r = 0.5762225745710553
Newspaper: r = 0.22829902637616534


All kind of ads are positively correlated with sales, so we can assume that higher ad expenses will lead to more sales. 
The TV ads have the strongest relationship with sales, while Newspaper ads have the weakest relationship with sales.

Ex 4 d)

I will predict the sales for the upcoming years, if the company will spend 50.000$ in TV ads.
First I will estimate the coefficients using the MLE from above and based on these make a prediction.
Afterwards I will evaluate the model. 

In [14]:
X = df[['TV', 'Radio', 'Newspaper']]
Y = df['Sales']
X.insert(0, 'Intercept', 1)

In [16]:
# Estimate the coefficients
beta = fit_model(X, Y)

# Print the regression coefficients and log-likelihood
print("Regression Coefficients:")
print(f"Intercept: {beta[0]}")
print(f"TV: {beta[1]}")
print(f"Radio: {beta[2]}")
print(f"Newspaper: {beta[3]}")

Regression Coefficients:
Intercept: 2.9388893694594316
TV: 0.04576464545539749
Radio: 0.18853001691820437
Newspaper: -0.0010374930424765783


In [17]:
def predict(X, coefficients):
        return coefficients[0] + np.dot(X, coefficients[1:])

In [18]:
X = [50, 0, 0]
predict(X, beta)

5.2271216422293065

Based on the estimated coefficient we would expect 5277 widgets to sell next year, if the company decides to spent 50.000$ on TV ads.

To evaluate the model, we first calculate the root mean squared error and the R^2

In [19]:
y_predicted = pd.DataFrame({"y_pred":predict(features, beta)})

In [20]:
def root_mean_squared_error(y_true, y_pred):

    # Convert inputs to NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate squared errors
    squared_errors = (y_true - y_pred) ** 2

    # Calculate mean squared error
    mean_squared_error = np.mean(squared_errors)

    # Calculate RMSE
    rmse = np.sqrt(mean_squared_error)

    return rmse

In [21]:
print("Range of response variable: From " + str(df["Sales"].min()) + " to " + str(df["Sales"].max()))

Range of response variable: From 1.6 to 27.0


In [22]:
root_mean_squared_error(response, y_predicted["y_pred"])

1.6685701407225697

It appears that the RMSE is relatively small compared to the range of the response variable. This suggests that my predictions have a relatively low average difference from the actual sales values.

In [23]:
def r_squared(y_true, y_pred):
    # Convert inputs to NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Calculate the sum of squared residuals
    ss_residuals = np.sum((y_true - y_pred) ** 2)

    # Calculate the total sum of squares
    ss_total = np.sum((y_true - np.mean(y_true)) ** 2)

    # Calculate R-squared
    r2 = 1 - (ss_residuals / ss_total)

    return r2

In [24]:
r_squared(response, y_predicted["y_pred"])

0.8972106381789522

The R-Squared shows that approximately 89.72% of tthe variance in the response variable (Sales) can be explained by my model. This suggests that my model is able to capture a significant portion if the variability in the data and provides a good fit.